# Вебинар 8. Консультация

---

## 1. Recap по финальному проекту

**Основное**
- Дедлайн - 4 октября 23:59
- Датасеты из вебинаров
- Целевая метрика money_precision@5
- Есть бизнес-ограничения
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть **ноутбук с решением** + **src с .py файлами** + файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000). **N > 3 000**
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, выручка от покупок,...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель (а это НЕ обязательно!) - следите за валидацией 

---

## 2. Ответы на вопросы

Как делать кросс-валидацию?
- [Статья](https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)
- Смотреть на метрику на отложенной выборке, меняя гиперпараметры

Какие гиперпараметры оптимизировать у ALS:
- Кол-во факторов 
- regularization / iterations

Как считать цены для товаров (в т ч для money_precison@5):
- Цена товара = среднее цен покупок этого товара по всем юзерам = sum(sales_value) / sum(quantity) из train

Новый пользователь:
- рекомендовать топ-полярных / random weighted топ-полярных товаров
- использовать гибридные рекомендательные системы

## Ваши вопросы

In [2]:
# questions?

# Вопрос 1. Перечитала методичку, там написано, что в implicit есть возможность перерасчёта холодных 
# юзеров "на лету". Как она реализована?

# псеводокод
model = AlterantingLeastSquares()
model.predict(..., recalculate_user=True) # Пересчет холодных юзеров на лету
# При обучении была матрица user-item
# В predict можете подать обновленную user-item матрицу (c новыми взаимодействиями)


# Вопрос 2. Все товары из разных категорий - как это сделать? 
# фильтровать выданные моделью рекомендации? - Да. Бейзлайн фильтрации есть в 7 вебинаре: postfilter_items()
# 2 новых товара и 1 дорогой - это аналогично постфильтрация? - Да

# Вопрос 3. а что делать если модель вернула меньше 5 рекомендаций? 
# Ответ 1: model.recommend_all() - ранжирует все товары. 
# https://implicit.readthedocs.io/en/latest/als.html#implicit.als.AlternatingLeastSquares.recommend_all

# Ответ 2: генерировать 200 рекомендаций --> фильтровать --> если останется < 5, то чем-то (популярными) дополнить

# Вопрос 4. precision считать уже после пост фильтрации? - ДА (после применения всех бизнес-правил)

# Вопрос 5. а можно по тестам какие-то примеры кода из жизни?
# Тесты на данные: great-expectations
# Тесты на predict: вариации assert. assert(len(predictions) == 5), ...
# Тесты на метрики: assert val_money_precision > 0.2, ...


#  Модель 1-ого уровня сильно проще, чем модель 2-ого уровня. 
# (ALS, логистическая регрессия, ближайшие соседи к покупкам юзера --> (LightGBM, Нейройнная сетка)

---

## 3. Разбор соревнования по рекомендациям топ-30 товаров X5 Retail Hero

Постановка задачи + особенности данных + подход к решению из [презентации](https://vk.com/away.php?utf=1&to=https%3A%2F%2Fgithub.com%2Faprotopopov%2Fretailhero_recommender%2Fblob%2Fmaster%2Fslides%2Fretailhero_recommender.pdf) решения 1-ого места

Бонус - [код](https://github.com/aprotopopov/retailhero_recommender) решения 1-ого места

In [11]:
import pandas as pd

# Item-Item matrix. a_ij - сколько раз товар i и товар j были куплены вместк
m = pd.DataFrame([[10, 3], [3, 6]], columns=['item_1', 'item_2'], index=['item_1', 'item_2'])
print(m, '\n')

# нормировка
for i in range(m.shape[0]):
    m[m.columns[i]] = m[m.columns[i]] / m.iloc[i, i]

print('normalized:')
print(m, '\n') 


# Статистика по товару
print(m.mean(axis=0))
print(m.min(axis=0))
print(m.std(axis=0))

        item_1  item_2
item_1      10       3
item_2       3       6 

normalized:
        item_1  item_2
item_1     1.0     0.5
item_2     0.3     1.0 

item_1    0.65
item_2    0.75
dtype: float64
item_1    0.3
item_2    0.5
dtype: float64
item_1    0.494975
item_2    0.353553
dtype: float64


## Основные подходы участников из топ-15

### Генерация кандидатов

В основном генерировали k = {50, 100} кандидатов, чтобы модель 2-ого уровня отрабатывала за разумное время

- Прошлые покупки юзера + топ-популярных, если покупок < k --> recall@100 ~ 42%
- Из ALS, item2item моделей (implicit.nn.CosineRecommender)

### Фичи в модели 2-ого уровня
Коллаборативные:
- biases + embeddings из коллаборативной фильтрации / Скоры item2item моделей
- TF-IDF матирца item-ов c > N покупками
- TF-IDF + TSNE/UMAP/ALS

handcrafted фичи для товаров:
- категории товаров
- нормированная частота покупки товара для каждого клиента = кол-во покупок товар i / общеее кол-во покупок юзера
- кол-во магазинов, в которых продавался товар
- кол-во транзакций клиента
- mean / max / std кол-ва уникальных товаров в корзине клиента 
- mean / max / std кол-ва уникальных категорий в корзине клиента 

handcrafted фичи для юзеров:
- Средний чек
- Средняя цена одного купленного товара
- Среднее кол-во дней между покупками / с последней покупки    
- Количество уникальных покупок (transaction_id, product_id) по всем категориям (store_id, level_i_id)
- Признаки с накопленными бонусами
- Средняя скидка, доля купленных товаров со скидками

Интересные:
- Факт заказа каждого товара в последних 5 транзакциях в виде последовательности бит (категориальная фича). 
10001 - купил товар в последней транзакции и 5 транзакий назад (feature hashing)
- item co_ocurrence
- als/word2vec эмбеддинги товаров (альтернативное название - item2vec, prod2vec)
- Расстояние от als/word2vec эмбеддинга товара до среднего эмбедиингов товаров, купленных юзером

ALS --> эмбеддигнги для всех товаров  
История покупок юзера --> товары юзера --> (ALS) эмбеддинги --> усредняем (1 средний эмбеддинг)

In [ ]:
# Работа с эмбеддингами товаров на примере ALS вместо word2vec
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares()
model.fit(...)

model.item_factors  # Добавить как фичи в модель 2-ого уровня

# Предположение: эмбеддинг юзера = среднее эмбеддингов купленниых им товаров
# эмбеддинг товара - эмбеддинг юзера  # чем разница меньше, тем товар ближе к интересам пользователя

### Модели 2-ого уровня
- классификация через LightGBM
- ранжирование CatBoost через YetiRank, YetiRankPairwise